# Import software libraries

In [157]:
# Import required libraries.
import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.
import pickle

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
[GCC 7.3.0]
- pandas 1.1.3
- sqlite3 3.30.1


# Examine the database

In [158]:
# Connect to SQLite database.
try: 
    conn = sqlite3.connect('/home/jovyan/work/Project/data/prod_sample.db')
    print("Connect to the database success")
except sqlite3.IntegrityError as e:
    print("ERROR OCCURED!")
    print(e)

cur = conn.cursor()

Connect to the database success


In [159]:
# List all the tables in the database.
cur.execute('''SELECT name
                FROM sqlite_master 
                WHERE type='table';
                ''')
print(cur.fetchall())

[('stock_description',), ('online_retail_history',)]


# Read data from the `online_retail_history` table

In [160]:
# Write the query to be executed that selects everything from the online_retail_history table.
query1 = '''SELECT *
            FROM online_retail_history;'''



# Use the read_sql function in pandas to read a query into a DataFrame.
online_retail_history = pd.read_sql(query1, conn)


# Preview the first five rows of the data.
online_retail_history.head(5)


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [161]:
# Get the shape of the data.
online_retail_history.shape


(15321, 8)

# Read data from the `stock_description` table

In [162]:
# Write the query to be executed that selects everything from the stock_description table.
query2 = '''SELECT *
            FROM stock_description;'''



# Use the read_sql function in pandas to read a query into a DataFrame.
stock_description = pd.read_sql(query2, conn)


# Preview the first five rows of the data.
stock_description.head(5)


,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [163]:
# Get the shape of the data.
stock_description.shape


(3952, 2)

# Aggregate the `online_retail_history` and `stock_description` datasets

In [164]:
# Write a query to aggregate the two datasets so that you have the stock descriptions as well as the stock code.
query_aggregate = '''SELECT StockCode, Description
                    FROM stock_description
                    GROUP BY StockCode'''


# Use the read_sql function in pandas to read a query into a DataFrame.
aggregate_data = pd.read_sql(query_aggregate, conn)


# Preview the first five rows of the data.
aggregate_data.head(5)


,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [165]:
# Get the shape of the data.
aggregate_data.shape


(3905, 2)

In [166]:
# Do a right join so users won't be lost.

hisory_w_description = \
online_retail_history.merge(stock_description,
                       on = 'StockCode', how = 'left')

hisory_w_description.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Identify and fix corrupt or unusable data

In [167]:
# Check the value counts of the "Description" field.
value_counts = hisory_w_description['Description'].value_counts()
value_counts

CREAM HANGING HEART T-LIGHT HOLDER    2174
JUMBO BAG RED RETROSPOT               1960
?                                     1711
REGENCY CAKESTAND 3 TIER              1711
PARTY BUNTING                         1615
LUNCH BAG RED RETROSPOT               1421
ASSORTED COLOUR BIRD ORNAMENT         1405
POPCORN HOLDER                        1329
LUNCH BAG  BLACK SKULL.               1271
SET OF 3 CAKE TINS PANTRY DESIGN      1257
PACK OF 72 RETROSPOT CAKE CASES       1178
Name: Description, dtype: int64

In [168]:
# Remove rows where "Description" is just a question mark (?).
clean_unuasable_data = hisory_w_description[hisory_w_description['Description'] != '?']


# Preview the first five rows of the data.
print (hisory_w_description.shape)
print (clean_unuasable_data.shape)
hisory_w_description_rows = hisory_w_description.shape[0]
clean_unuasable_data_rows = clean_unuasable_data.shape[0]

# Calculate the difference in row counts
difference = hisory_w_description_rows - clean_unuasable_data_rows

# Display the difference
print ("difference between before and after remove ? is {}".format(difference))
print ("you will see {} is ? marks form 'value_counts'".format(difference))

# check again 
# print(remove_mark_data)

# print to check removal is work or not
# print(value_counts==value_counts_after_remove_mark)

(17032, 9)
(15321, 9)
difference between before and after remove ? is 1711
you will see 1711 is ? marks form 'value_counts'


# Identify and remove duplicates

In [169]:
# Identify all duplicated data.
duplicated_data = clean_unuasable_data[clean_unuasable_data.duplicated(keep = False)]
print('Number of rows with duplicated data:', duplicated_data.shape[0])

Number of rows with duplicated data: 223


In [170]:
# Print the duplicated data.
duplicated_data

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
176,536863,20727,1,2010-12-03 11:19:00,1.65,u17967,United Kingdom,1.65,LUNCH BAG BLACK SKULL.
178,536863,20727,1,2010-12-03 11:19:00,1.65,u17967,United Kingdom,1.65,LUNCH BAG BLACK SKULL.
496,537781,84879,8,2010-12-08 12:46:00,1.69,u17341,United Kingdom,13.52,ASSORTED COLOUR BIRD ORNAMENT
497,537781,84879,8,2010-12-08 12:46:00,1.69,u17341,United Kingdom,13.52,ASSORTED COLOUR BIRD ORNAMENT
569,537955,20725,1,2010-12-09 11:28:00,1.65,u16782,United Kingdom,1.65,LUNCH BAG RED RETROSPOT
...,...,...,...,...,...,...,...,...,...
16673,580469,20727,1,2011-12-04 12:32:00,1.65,u14583,United Kingdom,1.65,LUNCH BAG BLACK SKULL.
16714,580611,20727,1,2011-12-05 11:49:00,1.65,u12748,United Kingdom,1.65,LUNCH BAG BLACK SKULL.
16716,580611,20727,1,2011-12-05 11:49:00,1.65,u12748,United Kingdom,1.65,LUNCH BAG BLACK SKULL.
16994,581449,22423,1,2011-12-08 17:37:00,12.75,u12748,United Kingdom,12.75,REGENCY CAKESTAND 3 TIER


In [171]:
# Remove the duplicated data.
clean_unusable_and_duplicate = clean_unuasable_data[~clean_unuasable_data.duplicated()]


# Preview the first five rows of the data.
print(clean_unusable_and_duplicate.head(5))
print(clean_unusable_and_duplicate.shape)
print(clean_unuasable_data.shape)


  Invoice StockCode  Quantity          InvoiceDate  Price CustomerID  \
0  536365    85123A         6  2010-12-01 08:26:00   2.55      u1785   
1  536367     84879        32  2010-12-01 08:34:00   1.69     u13047   
2  536373    85123A         6  2010-12-01 09:02:00   2.55      u1785   
3  536375    85123A         6  2010-12-01 09:32:00   2.55      u1785   
4  536378     20725        10  2010-12-01 09:37:00   1.65     u14688   

          Country  TotalAmount                         Description  
0  United Kingdom        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
1  United Kingdom        54.08       ASSORTED COLOUR BIRD ORNAMENT  
2  United Kingdom        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
3  United Kingdom        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
4  United Kingdom        16.50             LUNCH BAG RED RETROSPOT  
(15206, 9)
(15321, 9)


# Correct date formats

In [172]:
# Get the data types for every column in the DataFrame.
clean_unusable_and_duplicate.info()

cleaned_data = clean_unusable_and_duplicate.copy()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15206 entries, 0 to 17031
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Invoice      15206 non-null  object 
 1   StockCode    15206 non-null  object 
 2   Quantity     15206 non-null  int64  
 3   InvoiceDate  15206 non-null  object 
 4   Price        15194 non-null  float64
 5   CustomerID   12435 non-null  object 
 6   Country      15206 non-null  object 
 7   TotalAmount  15194 non-null  float64
 8   Description  15206 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 1.2+ MB


In [173]:
# Convert "InvoiceDate" to a "%Y-%m-%d" datetime format.
cleaned_data['InvoiceDate'] = pd.to_datetime(cleaned_data['InvoiceDate'],format = '%Y-%m-%d')



In [174]:
# Get the data types for every column in the converted DataFrame.
cleaned_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15206 entries, 0 to 17031
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Invoice      15206 non-null  object        
 1   StockCode    15206 non-null  object        
 2   Quantity     15206 non-null  int64         
 3   InvoiceDate  15206 non-null  datetime64[ns]
 4   Price        15194 non-null  float64       
 5   CustomerID   12435 non-null  object        
 6   Country      15206 non-null  object        
 7   TotalAmount  15194 non-null  float64       
 8   Description  15206 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 1.2+ MB


# Examine the table before finishing

In [175]:
# Preview the first five rows of the data.
cleaned_data.head(5)


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Load the dataset into a pickle file

In [181]:
# Save the dataset as a pickle file named online_history_cleaned.pickle.
cleaned_data.to_pickle('online_history_cleaned.pickle')


In [182]:
# Close any connections to the database.
conn.close()
